### Problem Objective

The goal of this problem is to create a grid, filled with buildings, roads, and empty spaces. The building cells must be contiguous and share the same ID. We also need to construct a graph based on the grid, where connections between roads and buildings are represented as edges, and the width of the roads defines the weight of those edges.

#### How it works:
- I chose the BFS (Breadth-First Search) algorithm to identify and mark cells belonging to the same building. 

- From the grid, we build a graph where vertices represent intersections and buildings, and edges represent connections between buildings. The weight of each edge is determined by the width of the road connecting the buildings.



In [9]:
import random
from collections import defaultdict, deque

In [10]:
# Let's define the dimensions of the grid 

I = 50 # rows
J = 50 # cols

In [11]:
# Func to calc the road width

width_cache = {}

def calc_road_w(x, y, grid):
    """This func calculates the road width."""
    if (x, y) in width_cache:
        return width_cache[(x, y)]

    visited = set()
    queue = [(x, y)]
    width = 0

    if grid[x][y] == "ROAD":
        while queue:
            cx, cy = queue.pop()
            if (cx, cy) not in visited:
                visited.add((cx, cy))
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    nx, ny = cx + dx, cy + dy
                    if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == "ROAD" and (nx, ny) not in visited:
                        queue.append((nx, ny))
        width = len(visited)

    width_cache[(x, y)] = width
    return width


In [12]:
def bfs_building(x, y, grid, building_id):
    """BFS to connect the buildings that have the same id."""
    visited = set()
    queue = deque([(x, y)])
    building_cells = []

    while queue:
        cx, cy = queue.popleft()
        if (cx, cy) not in visited and grid[cx][cy] == building_id:
            visited.add((cx, cy))
            building_cells.append((cx, cy))
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = cx + dx, cy + dy
                if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == building_id:
                    queue.append((nx, ny))
    return building_cells

In [13]:

def create_graph(grid, warehouses):
    """Generate the graph."""
    graph = defaultdict(list)
    vertices = {}
    vertex_id = 0

    def add_vertex(x, y, type_, building_id=None):
        nonlocal vertex_id
        if (x, y) not in vertices:
            vertices[(x, y)] = vertex_id
            graph[vertex_id] = {"type": type_,
                                "edges": [], "building_id": building_id}
            vertex_id += 1
        return vertices[(x, y)]

    building_id_map = {}

    for x, row in enumerate(grid):
        for y, cell in enumerate(row):
            if cell == "ROAD":
                width = calc_road_w(x, y, grid)
                add_vertex(x, y, "ROAD")
            elif isinstance(cell, int):
                if cell in warehouses:
                    add_vertex(x, y, "WAREHOUSE")
                else:
                    if cell not in building_id_map:
                        building_cells = bfs_building(x, y, grid, cell)
                        building_vertex = add_vertex(
                            building_cells[0][0], building_cells[0][1], "BUILDING", building_id=cell)
                        building_id_map[cell] = building_vertex
                        for cell_x, cell_y in building_cells:
                            add_vertex(cell_x, cell_y, "BUILDING",
                                       building_id=cell)
                    else:
                        add_vertex(x, y, "BUILDING", building_id=cell)

    for (x1, y1), v1 in vertices.items():
        for (x2, y2), v2 in vertices.items():
            if (x1 == x2 and abs(y1 - y2) == 1) or (y1 == y2 and abs(x1 - x2) == 1):
                width = calc_road_w(x1, y1, grid)
                if grid[x2][y2] == "ROAD":
                    graph[v1]["edges"].append((v2, width))
                    graph[v2]["edges"].append((v1, width))
                elif isinstance(grid[x2][y2], int) and grid[x2][y2] != "ROAD":
                    graph[v1]["edges"].append((v2, width))
                    graph[v2]["edges"].append((v1, width))

    return graph, vertices

In [14]:
def create_graphviz(graph):
    """Generate the Graphviz code to visualize the graph."""
    dot = "graph G {\n"
    for v, info in graph.items():
        label = info["type"]
        building_id = info["building_id"]
        if building_id is not None:
            # Exibe o ID do prédio como o rótulo
            label = f"Building {building_id}"
        dot += f'    {v} [label="{label}"];\n'
    for v, info in graph.items():
        for neighbor, width in info["edges"]:
            if v < neighbor:
                dot += f'    {v} -- {neighbor} [label="Width: {width}"];\n'
    dot += "}"
    return dot

In [15]:
def create_random_grid(rows, cols, rp=0.6, bp=0.35, wp=0.05):
    """Generates a random grid."""
    grid = []
    warehouses = set()

    for _ in range(rows):
        row = []
        for _ in range(cols):
            rand = random.random()
            if rand < rp:
                row.append("ROAD")
            elif rand < rp + bp:
                building_id = random.randint(1, 5000)
                row.append(building_id)
            else:
                warehouse_id = random.randint(0, 10)
                row.append(warehouse_id)
                warehouses.add(warehouse_id)
        grid.append(row)

    return grid, warehouses

In [16]:

grid, warehouses = create_random_grid(I, J)
graph, vertices = create_graph(grid, warehouses)

graphviz = create_graphviz(graph)


with open("graphviz.txt", "w", encoding="utf-8") as f:
    f.write(graphviz)